## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-04-02-44-38 +0000,nypost,California sex traffic crackdown leads to 611 ...,https://nypost.com/2026/02/03/us-news/californ...
1,2026-02-04-02-41-11 +0000,nypost,Attorney defending accused $23M homeless fraud...,https://nypost.com/2026/02/03/us-news/alleged-...
2,2026-02-04-02-40-55 +0000,nypost,Fellow DSA member wins special election to fil...,https://nypost.com/2026/02/03/us-news/fellow-d...
3,2026-02-04-02-39-00 +0000,wsj,New York and New Jersey Sue Administration for...,https://www.wsj.com/us-news/new-york-and-new-j...
4,2026-02-04-02-33-04 +0000,startribune,Federal judge orders release of two men in ICE...,https://www.startribune.com/mn-venezuelans-rel...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2529/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
50,trump,40
166,epstein,22
31,new,19
309,house,17
44,ice,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
289,2026-02-03-14-44-00 +0000,wsj,Bill and Hillary Clinton made a new offer to g...,https://www.wsj.com/politics/policy/bill-hilla...,118
63,2026-02-04-00-00-00 +0000,wsj,Stephen Miller called Alex Pretti a “domestic ...,https://www.wsj.com/politics/policy/ice-stephe...,112
38,2026-02-04-01-07-00 +0000,wsj,Fed governor Stephen Miran said he resigned fr...,https://www.wsj.com/economy/central-banking/st...,104
115,2026-02-03-22-17-00 +0000,wsj,President Trump shared a rendering Tuesday of ...,https://www.wsj.com/politics/policy/trump-whit...,101
87,2026-02-03-22-56-00 +0000,wsj,The Republican-led House passed funding legisl...,https://www.wsj.com/politics/policy/government...,99


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
289,118,2026-02-03-14-44-00 +0000,wsj,Bill and Hillary Clinton made a new offer to g...,https://www.wsj.com/politics/policy/bill-hilla...
63,95,2026-02-04-00-00-00 +0000,wsj,Stephen Miller called Alex Pretti a “domestic ...,https://www.wsj.com/politics/policy/ice-stephe...
38,71,2026-02-04-01-07-00 +0000,wsj,Fed governor Stephen Miran said he resigned fr...,https://www.wsj.com/economy/central-banking/st...
181,62,2026-02-03-20-03-43 +0000,cbc,Disney parks chief Josh D'Amaro replaces Bob I...,https://www.cbc.ca/news/entertainment/disney-c...
258,61,2026-02-03-16-19-18 +0000,bbc,Son of Norway's crown princess hears details o...,https://www.bbc.com/news/articles/ckgn8dnxjgwo...
40,53,2026-02-04-00-54-23 +0000,cbc,French police raid X offices as they investiga...,https://www.cbc.ca/news/world/french-police-ra...
4,50,2026-02-04-02-33-04 +0000,startribune,Federal judge orders release of two men in ICE...,https://www.startribune.com/mn-venezuelans-rel...
50,44,2026-02-04-00-31-15 +0000,nypost,Netflix co-CEO grilled by senators as Warner B...,https://nypost.com/2026/02/03/business/netflix...
178,42,2026-02-03-20-11-18 +0000,nypost,Ukrainian husband weeps beside childhood sweet...,https://nypost.com/2026/02/03/world-news/ukrai...
186,40,2026-02-03-19-53-14 +0000,startribune,Tim Pawlenty: The crisis will subside in Minne...,https://www.startribune.com/former-mn-gov-ice-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
